In [31]:
import numpy as np
import joblib
import os
from pathlib import Path
import tensorflow as tf
from Utils.Data_Preprocessing import *
from sklearn.preprocessing import MinMaxScaler

def load_models():
    """
    Load both .pkl and .h5 models from the Models directory.
    Simply loads and prints model types without doing any inference.
    """
    # Define path to your Models directory
    project_root = Path('.')  # Current directory
    models_dir = project_root / 'Models'
    
    # Check if directory exists
    if not models_dir.exists():
        print(f"Error: Models directory not found at {models_dir}")
        return None
    
    print(f"Loading models from: {models_dir}")
    
    # Define model files based on your directory structure
    pkl_models = [
        'decision_tree_model.pkl',
        'linear_regression_model.pkl',
        'prophet_model.pkl',
        'random_forest_model.pkl',
        'slinear_regression_model.pkl',
        'svr_model.pkl',
        'voting_model.pkl',
        'xgboost_model.pkl'
    ]
    
    h5_models = [
        'gru_model.h5',
        'lstm_model.h5'
    ]
    
    # Dictionary to store loaded models
    loaded_models = {}
    
    # Load pickle models
    print("\n--- Loading Pickle Models ---")
    for model_file in pkl_models:
        model_path = models_dir / model_file
        try:
            # Only try to load if file exists
            if model_path.exists():
                model = joblib.load(model_path)
                model_name = model_path.stem  # Get filename without extension
                loaded_models[model_name] = model
                
                # Check model type
                model_type = type(model).__name__
                has_predict = hasattr(model, 'predict')
                
                print(f"✓ {model_name}: Type={model_type}, Has predict method={has_predict}")
            else:
                print(f"⚠ {model_file} not found, skipping")
        except Exception as e:
            print(f"✗ Error loading {model_file}: {e}")
    
    # Load h5 models (TensorFlow/Keras models)
    print("\n--- Loading H5 Models ---")
    for model_file in h5_models:
        model_path = models_dir / model_file
        try:
            # Only try to load if file exists
            if model_path.exists():
                # TensorFlow requires string path
                model = tf.keras.models.load_model(str(model_path))
                model_name = model_path.stem  # Get filename without extension
                loaded_models[model_name] = model
                
                # Check model type and summary
                model_type = type(model).__name__
                has_predict = hasattr(model, 'predict')
                
                print(f"✓ {model_name}: Type={model_type}, Has predict method={has_predict}")
                
                # Print model summary
                print(f"Model structure:")
                #model.summary()
            else:
                print(f"⚠ {model_file} not found, skipping")
        except Exception as e:
            print(f"✗ Error loading {model_file}: {e}")
    
    print(f"\nSuccessfully loaded {len(loaded_models)} models")
    return loaded_models

In [32]:
def inference_classical_models(data):
    import numpy as np
    import pandas as pd
    import joblib

    # Load the scalers
    scaler_X = joblib.load('Scaler/scaler_X_minmax.pkl')
    scaler_y = joblib.load('Scaler/scaler_y_minmax.pkl')
    #model = joblib.load('Models/linear_regression_model.pkl')
    
    loaded_models= load_models()
    
    # Filter only classical models (skip keras models which are of type 'Functional', 'Sequential', etc.)
    classical_models = {
        name: model for name, model in loaded_models.items()
        if not isinstance(model, tf.keras.Model)
    }

    # # Read last 20 rows (not 19!) for moving average support
    # df_hist = pd.read_csv('Data/Tasla_Stock_Updated_V2.csv').tail(20)

    # # Add the new data (must match expected column names)
    # df_new = pd.DataFrame([data], columns=['Low', 'High', 'Open', 'Close', 'Volume'])
    # df_combined = pd.concat([df_hist, df_new], ignore_index=True)
    
    # # Select features for prediction (same as used during training)
    # features = ['Low', 'High', 'Open', 'Close', 'Volume']
    # # Extract the last row (new data) with features
    # X = df_combined[features].iloc[-1:].values  # Shape: (1, n_features)

    # Scale the features
    X_scaled = scaler_X.transform(np.array(data).reshape(1, -1))

    predictions = {}

    # Inference using each classical model
    for name, model in classical_models.items():
        try:
            y_pred_scaled = model.predict(X_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
            predictions[name] = y_pred[0][0]
            print(f"✓ {name} prediction: {y_pred[0][0]:.4f}")
        except Exception as e:
            print(f"✗ Error predicting with {name}: {e}")

    return predictions
    


inference_classical_models([23.543333053588867,23.83133316040039,22.858667373657227,22.99933242797852,114088500])

Loading models from: Models

--- Loading Pickle Models ---
✓ decision_tree_model: Type=DecisionTreeRegressor, Has predict method=True
✓ linear_regression_model: Type=LinearRegression, Has predict method=True
✓ prophet_model: Type=Prophet, Has predict method=True
✓ random_forest_model: Type=RandomForestRegressor, Has predict method=True
⚠ slinear_regression_model.pkl not found, skipping
✓ svr_model: Type=SVR, Has predict method=True
✓ voting_model: Type=VotingRegressor, Has predict method=True
✓ xgboost_model: Type=XGBRegressor, Has predict method=True

--- Loading H5 Models ---


✓ gru_model: Type=Sequential, Has predict method=True
Model structure:
✓ lstm_model: Type=Sequential, Has predict method=True
Model structure:

Successfully loaded 9 models
✓ decision_tree_model prediction: 20.1507
✓ linear_regression_model prediction: 23.3113
✗ Error predicting with prophet_model: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
✓ random_forest_model prediction: 22.9479
✓ svr_model prediction: 46.4437
✓ voting_model prediction: 28.9141
✓ xgboost_model prediction: 22.9535


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/

{'decision_tree_model': 20.15066719055176,
 'linear_regression_model': 23.311288549092335,
 'random_forest_model': 22.947899913787843,
 'svr_model': 46.44371812038351,
 'voting_model': 28.914106112508616,
 'xgboost_model': 22.953518}